In [ ]:
def convert2matrix(data_arr, look_back):
   X, Y =[], []
   for i in range(len(data_arr)-look_back):
     d=i+look_back
     X.append(data_arr[i:d,])
     Y.append(data_arr[d,])
   return np.array(X), np.array(Y)

def model_lstm_uni(look_back):
    tf.keras.backend.clear_session()
    model=Sequential()
    model.add(LSTM(100, input_shape=(1, look_back), activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model

from sklearn.model_selection import TimeSeriesSplit
def forecast_error(data, first_rem=0.5):
    train, test = train_test_split(data, train_size=first_rem, shuffle=False)
    look_back = 5
    trainX, trainY = convert2matrix(np.array(train), look_back)
    testX, testY = convert2matrix(np.array(test), look_back)
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    model=model_lstm_uni(look_back)
    history = model.fit(trainX,
                      trainY,
                      epochs=100,
                      batch_size=30,
                      validation_data=(testX, testY),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=10)],
                      verbose=0,
                      shuffle=False)

    train_predict = model.predict(trainX)
    test_predict = model.predict(testX)
    error = (np.sqrt(mean_squared_error(testY, test_predict)))

    return error,trainX, testX,testY, test_predict

def multi_data_prep(s1, s2):
    # define input sequence
    in_seq1 = array(s1[:-1])
    in_seq2 = array(s2[:-1])
    out_seq = array(s1[1:])
    # convert to [rows, columns] structure
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2, out_seq))

    return dataset

def multi_data_prep_three(s1, s2,s3):
    # define input sequence
    in_seq1 = array(s1[:-1])
    in_seq2 = array(s2[:-1])
    in_seq3 = array(s3[:-1])
    out_seq = array(s1[1:])
    # convert to [rows, columns] structure
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    in_seq3 = in_seq3.reshape((len(in_seq3), 1))
    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2, in_seq3,out_seq))

    return dataset

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

#Multivariate model
#Multivariate data preparation

def forecast_error_multi(data, n_features, first_rem=0.5):
    train, test = train_test_split(data, train_size=first_rem, shuffle=False)
  # choose a number of time steps
    n_steps = 5
  # convert into input/output
    trainX, trainY = split_sequences(train, n_steps)
    testX, testY = split_sequences(test, n_steps)

    # the dataset knows the number of features, e.g. 2
    #n_features = 2
    # define model
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=100, verbose=0)

    train_predict = model.predict(trainX)
    test_predict = model.predict(testX)
    error = (np.sqrt(mean_squared_error(testY, test_predict)))

    print(error)
    return error,trainX, testX, testY, test_predict

def calculations_cgc(inputs):
    df,input_bands = inputs
    #print(list_of_bands)
    list_of_bands = pd.DataFrame(input_bands).reset_index()

    #for index, list_of_bands in enumerate(input_bands):
    for i in range(0, len(list_of_bands)):
        x_band = list_of_bands['X'][i]
        y_band = list_of_bands['Y'][i]
        z_band = list_of_bands['Z'][i]

        print(x_band, y_band, z_band)


        two_sig_data = multi_data_prep(df[x_band], df[y_band])
        three_sig_data = multi_data_prep_three(df[x_band], df[y_band], df[z_band])

        error1, x_train1, x_test1, y_test1, y_hat1 = forecast_error_multi(two_sig_data, 2)
        error2, x_train2, x_test2, y_test2, y_hat2 = forecast_error_multi(three_sig_data, 3)

        cgc = (error1 - error2)
        percentage = (cgc/error1)*100
        list_of_bands['XY_X_Error'][i] = error1
        list_of_bands['XYZ_X_Error'][i] = error2
        list_of_bands['CGC_Value'][i] = cgc
        list_of_bands['Percentage Reduction'][i] = percentage

    return list_of_bands



def calculations_gc(inputs):
    df,input_bands = inputs
    #print(list_of_bands)
    list_of_bands = pd.DataFrame(input_bands).reset_index()

    #for index, list_of_bands in enumerate(input_bands):
    for i in range(0, len(list_of_bands)):
        x_band = list_of_bands['X'][i]
        y_band = list_of_bands['Y'][i]

        print(x_band, y_band)


        two_sig_data = multi_data_prep(df[x_band], df[y_band])

        error1, x_train1, x_test1, y_test1, y_hat1 = forecast_error(df[x_band])
        error2, x_train2, x_test2, y_test2, y_hat2 = forecast_error_multi(two_sig_data, 2)

        gc = (error1 - error2)
        percentage = (gc/error1)*100
        list_of_bands['X_X_Error'][i] = error1
        list_of_bands['XY_X_Error'][i] = error2
        list_of_bands['GC_Value'][i] = gc
        list_of_bands['Percentage Reduction'][i] = percentage


    return list_of_bands